# **Feature Selection**

## Objectives
* Do a grid search using cross-validation in order to select a classification model. We will run this grid search several times removing different features in order to test our Hytpothesis # something and construct non-trivial models.

## Inputs
* The train and test data set aside in the the last notebook.
* The pipeline that was produced in the last notebook.
* Parameter values determined in previous notebook.

## Outputs
* A choice of classification model that we will further tune and evaluate.

## Additional Comments
* We have chosen to do classification. We may yet do a regression model on the point differential.      

---

# Change working directory
We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os

home_dir = '/workspace/pp5-ml-dashboard'
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)

/workspace/pp5-ml-dashboard


We now load our training and test sets, as well as some of the packages that we will be using.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from src.utils import get_df, save_df

train_dir = 'train/csv'
X_TrainSet = get_df('X_TrainSet',train_dir)
Y_TrainSet = get_df('Y_TrainSet',train_dir)

test_dir = 'test/csv'
X_TestSet = get_df('X_TestSet',test_dir)
Y_TestSet = get_df('Y_TestSet',test_dir)

## Section 1: Full Pipeline
We will build the full pipeline here. It will accept some parameters so that we can tune it later. We also declare some constants that we established in the last notebook.

In [3]:
from sklearn.preprocessing import StandardScaler
from feature_engine import transformation as vt
from feature_engine.selection import DropFeatures, SmartCorrelatedSelection
from sklearn.pipeline import Pipeline


# Constants needed for feature engineering
META = ['season', 'play_off']
TRIVIAL = ['pts_home','pts_away','plus_minus_home']
THRESH = 0.85
TRANSFORMS = {'log_e':(vt.LogTransformer, False), 
                'log_10':(vt.LogTransformer,'10'),
                'reciprocal':(vt.ReciprocalTransformer,False), 
                'power':(vt.PowerTransformer,False),
                'box_cox':(vt.BoxCoxTransformer,False),
                'yeo_johnson':(vt.YeoJohnsonTransformer,False)}
TRANSFORM_ASSIGNMENTS = {
    'yeo_johnson': ['dreb_away', 'blk_home', 'oreb_away', 'fta_away', 'dreb_home', 
                    'ast_home', 'stl_away', 'pts_away', 'stl_home', 'reb_away',
                    'pts_home', 'fgm_away', 'oreb_home', 'pf_away', 'pf_home'],
    'box_cox': ['ast_away', 'fta_home']
                            }


def base_pipeline(to_drop=None,thresh=THRESH):
    if not to_drop:
        to_drop = META
    else:
        to_drop.extend(META)
    to_drop = list(set(to_drop))
    pipeline = Pipeline([
        ('dropper', DropFeatures(features_to_drop=to_drop)),
        ('corr_selector', SmartCorrelatedSelection(method="pearson",
                                                   threshold=thresh,
                                                   selection_method="variance")
                                                   )
                        ])
    pipeline.to_drop = to_drop
    return pipeline

    
def add_transformations(pipeline, transform_assignments):
    # This needs to be called after the above is fit so that the correlation selector has that attr
    to_drop = pipeline.to_drop
    dropping = set(to_drop + pipeline['corr_selector'].features_to_drop_)
    
    new_assignments = { key: [val for val in value if val not in dropping] 
                       for key,value in transform_assignments.items()}
    ''' Olde version
    for value in transform_assignments.values():
        for drop_term in dropping:
            if drop_term in value:
                value.remove(drop_term)
    '''
    for transform, targets in new_assignments.items():
        if not targets:
            continue
        pipeline.steps.append(
            (transform, TRANSFORMS[transform][0](variables=targets))
            )
    pipeline.steps.append(('scaler', StandardScaler()))
    return pipeline

We add the last two steps of the pipeline, the feature selection and the model itself. We do this in a separate step since it depends on the model.

After a first pass, we will see that our models only use the features in `TRIVIAL`. These features make the classification problem too ... trivial as the winner of the game can be computed directly from `'plus_minus_home'`. The points of the each team can be used together to compute the winner as well, so we also remove them. 

# Attention
this could probably be implimented better with classes, but I think I don't want to worry too much about inheritance, I could do it with composition, and then tune the hyperparameters via methods. Do that later.

In [4]:
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

MODELS = {
    'LogisticRegression': LogisticRegression,
    'DecisionTree': DecisionTreeClassifier,
    'RandomForest': RandomForestClassifier,
    'GradientBoosting': GradientBoostingClassifier,
    'ExtraTrees': ExtraTreesClassifier,
    'AdaBoost': AdaBoostClassifier,
    'XGBoost': XGBClassifier
}


def add_feat_selection_n_model(pipeline,model_name,random_state=42):
    model = MODELS[model_name](random_state=random_state)
    pipeline.steps.append(("feat_selection", SelectFromModel(model)))
    pipeline.steps.append((model_name,model))
    return pipeline

Let us now create our list of models that we are going to train.

In [5]:
def create_pipelines(to_drop=None,thresh=THRESH):
    PIPELINES = {}
    for model_name in MODELS:
        base_pipe = base_pipeline(to_drop,thresh)
        base_pipe.fit(X_TrainSet)
        pipe_w_transforms= add_transformations(base_pipe,TRANSFORM_ASSIGNMENTS)
        PIPELINES[model_name] = add_feat_selection_n_model(pipe_w_transforms,model_name)
    return PIPELINES
PIPELINES = create_pipelines(thresh=0.6)

## Section 2: Cross Validation Search
We are going to perform multiple grid searchs as we refine our selection of features. The final grid search will determine the best model for our data, which we will tune in the next notebook.

We have a few different Hypotheses. One of them, hypothesis 1, is that the models will gravitate strongly towards the features related to points. In order of estimated impact, they are:
* `'plus_minus'`
* `'pts'`
* `'fgm'`
* `'fg3m'`, `'ftm'`

We will start by training our model on all of these features. To test our hypothesis, we will remove one collection of features at a time until we are left with no features that help you directly determine the score (and hence winner). We will check which features the models use along the way to see if we can validate our hypothesis. We expect that the earlier models will be able to predict the outcome of the games flawlessly.

We won't be passing any parameters as we will tune the hyperparameters in the following notebook.

Note: we are silencing warnings below. These warnings generate  hundreds of lines and no exceptions are raised.

In [6]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import logging
logging.captureWarnings(True)

In [8]:
from sklearn.model_selection import GridSearchCV
# to suppress warnings, I think
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import logging
logging.captureWarnings(True)
os.environ['PYTHONWARNINGS']='ignore'
#from sklearn.utils.testing import ignore_warnings

#@ignore_warnings(category=FutureWarning)
def grid_search(X_train, y_train, pipelines,param_grid={}):
    GRIDS = {}
    for pipe_name, pipe in pipelines.items():
        print(f"### Beginning grid search for {pipe_name} ###")
        try:
            grid=GridSearchCV(estimator=pipe,
                    param_grid=param_grid,
                    cv=5,
                    n_jobs=-2,
                    verbose=3,
                    scoring='accuracy')
            grid.fit(X_train,y_train)
        except ValueError as e:
            msg = "Invalid parameter model for estimator"
            if msg not in str(e):
                raise e
            print("An estimator was passed invalid parameters. We will "
                  f"continue the grid search for {pipe_name} with "
                  "param_grid=\{\}")
            print(f"### Beginning grid search for {pipe_name} with empty params. ###")
            grid=GridSearchCV(estimator=pipe,
                    param_grid={},
                    cv=5,
                    n_jobs=-2,
                    verbose=3,
                    scoring='accuracy')
            grid.fit(X_train,y_train)
        GRIDS[pipe_name] = grid
    return GRIDS
GRIDS = grid_search(X_TrainSet,Y_TrainSet,PIPELINES)

### Beginning grid search for LogisticRegression ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWa

[CV 4/5] END ..................................., score=1.000 total time=   0.9s
[CV 3/5] END ..................................., score=1.000 total time=   0.9s
[CV 5/5] END ..................................., score=1.000 total time=   0.9s
[CV 2/5] END ..................................., score=1.000 total time=   0.9s
[CV 1/5] END ..................................., score=1.000 total time=   1.0s
### Beginning grid search for DecisionTree ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWa

[CV 5/5] END ..................................., score=1.000 total time=   0.7s
[CV 2/5] END ..................................., score=1.000 total time=   0.8s
[CV 4/5] END ..................................., score=1.000 total time=   0.8s
[CV 3/5] END ..................................., score=1.000 total time=   0.8s
[CV 1/5] END ..................................., score=1.000 total time=   0.7s
### Beginning grid search for RandomForest ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

[CV 5/5] END ..................................., score=1.000 total time=   6.1s
[CV 3/5] END ..................................., score=1.000 total time=   5.9s
[CV 2/5] END ..................................., score=1.000 total time=   6.2s
[CV 4/5] END ..................................., score=1.000 total time=   5.7s
[CV 1/5] END ..................................., score=1.000 total time=   6.1s
### Beginning grid search for AdaBoost ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWa

[CV 1/5] END ..................................., score=1.000 total time=   1.0s
[CV 4/5] END ..................................., score=1.000 total time=   0.9s
[CV 2/5] END ..................................., score=1.000 total time=   0.8s
[CV 5/5] END ..................................., score=1.000 total time=   1.0s
[CV 3/5] END ..................................., score=1.000 total time=   0.8s
### Beginning grid search for XGBoost ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.In

[CV 1/5] END ..................................., score=1.000 total time=   2.6s
[CV 3/5] END ..................................., score=1.000 total time=   2.7s
[CV 2/5] END ..................................., score=1.000 total time=   2.7s
[CV 4/5] END ..................................., score=1.000 total time=   2.8s
[CV 5/5] END ..................................., score=1.000 total time=   2.8s


We see from the above that the models are "flawless." The reports in

We will use the following code to generate reports for evaluating our models. You can pass one of the resulting pipelines to `'clf_performance'` or the whole grid to `'grid_search_report_best'` if you like.

In [9]:
from src.model_eval import clf_performance, get_best_scores, grid_search_report_best

print(GRIDS.keys())
def extract_best_estimator(grid_collection, model_name):
  return grid_collection[model_name].best_estimator_

get_best_scores(GRIDS)

grid_search_report_best(GRIDS,X_TrainSet,Y_TrainSet,X_TestSet,Y_TestSet,['loss','win'])


dict_keys(['LogisticRegression', 'DecisionTree', 'RandomForest', 'GradientBoosting', 'ExtraTrees', 'AdaBoost', 'XGBoost'])
The best performing LogisticRegression model had a score of 100.0%.

The best performing DecisionTree model had a score of 100.0%.

The best performing RandomForest model had a score of 100.0%.

The best performing GradientBoosting model had a score of 100.0%.

The best performing ExtraTrees model had a score of 100.0%.

The best performing AdaBoost model had a score of 100.0%.

The best performing XGBoost model had a score of 100.0%.

LogisticRegression
#### Train Set #### 

---  Confusion Matrix  ---
                Actual loss Actual win
Prediction loss       13689          0
Prediction win            0      21187


---  Classification Report  ---
              precision    recall  f1-score   support

        loss       1.00      1.00      1.00     13689
         win       1.00      1.00      1.00     21187

    accuracy                           1.00     34876


However, we will not look at the reports for each model. There was no model that correctly classified a single game. This is as we expected.
The points scored in the game determine who wins, and we have a lot of that data still present. Let's recall which features we train on. Some features that we dropped were common to each model.

In [11]:
print(META)
sample_pipe = GRIDS['LogisticRegression'].best_estimator_
corr_dropped = list(sample_pipe['corr_selector'].features_to_drop_)


['season', 'play_off']


So some of the point related features were removed. But plus/minus score and actual scores were still retained.
In the end, each model was only trained an the following features.

In [12]:
auto_dropped = META + corr_dropped
remaining_cols = [col for col in X_TrainSet.columns if col not in auto_dropped]

def find_features(X,fitted_pipe):
    features = fitted_pipe['feat_selection'].get_support()
    X = X_TrainSet.filter(remaining_cols)
    if len(X.columns) != features.shape[0]:
        raise ValueError
    for index, col in enumerate(X.columns):
        if not features[index]:
            X.drop(col, axis=1, inplace=True)
    return X.columns

best_pipes = {name: grid.best_estimator_ for name, grid in GRIDS.items()}

for pipe_name, pipe in best_pipes.items():
    print(pipe_name)
    print(find_features(X_TrainSet,pipe))

LogisticRegression
Index(['pts_home', 'plus_minus_home', 'pts_away'], dtype='object')
DecisionTree
Index(['plus_minus_home'], dtype='object')
RandomForest
Index(['pts_home', 'plus_minus_home', 'pts_away'], dtype='object')
GradientBoosting
Index(['plus_minus_home'], dtype='object')
ExtraTrees
Index(['pts_home', 'plus_minus_home', 'pts_away'], dtype='object')
AdaBoost
Index(['plus_minus_home'], dtype='object')
XGBoost
Index(['plus_minus_home'], dtype='object')


This a large step towards validating our first hypothesis. Let's remove `'plus_minus_home'` and see how this impacts the models.

In [13]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import logging
logging.captureWarnings(True)
import sys
os.environ['PYTHONWARNINGS']='ignore'

PIPELINES_wo_pm = create_pipelines(to_drop=['plus_minus_home'],thresh=0.6)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    GRIDS_wo_pm = grid_search(X_TrainSet,Y_TrainSet,PIPELINES_wo_pm)

### Beginning grid search for LogisticRegression ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWa

[CV 1/5] END ..................................., score=1.000 total time=   1.0s
[CV 2/5] END ..................................., score=1.000 total time=   1.1s


/workspace/.pip-modules/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/workspace/.pip-modules/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/workspace/.pip-modules/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ..................................., score=1.000 total time=   1.0s
[CV 3/5] END ..................................., score=1.000 total time=   1.1s
[CV 5/5] END ..................................., score=1.000 total time=   1.0s
### Beginning grid search for DecisionTree ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

[CV 3/5] END ..................................., score=1.000 total time=   7.3s
[CV 4/5] END ..................................., score=1.000 total time=   7.4s
[CV 1/5] END ..................................., score=1.000 total time=   7.5s
[CV 5/5] END ..................................., score=1.000 total time=   7.5s
[CV 2/5] END ..................................., score=1.000 total time=   7.5s
### Beginning grid search for AdaBoost ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  f = X[feature_group].std().sort_values(ascending=False).index[0]
/workspace/.pip-modules/lib/python3.8/site-packages/feature_engine/selection/smart_correlation_selection.py:271: FutureWa

[CV 1/5] END ..................................., score=0.988 total time=   4.0s
[CV 2/5] END ..................................., score=0.986 total time=   4.0s
[CV 4/5] END ..................................., score=0.987 total time=   4.0s
[CV 3/5] END ..................................., score=0.989 total time=   4.1s
[CV 5/5] END ..................................., score=0.988 total time=   4.1s
### Beginning grid search for XGBoost ###
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/workspace/.pip-modules/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.In

[CV 1/5] END ..................................., score=1.000 total time=   6.1s
[CV 5/5] END ..................................., score=1.000 total time=   6.2s
[CV 2/5] END ..................................., score=0.999 total time=   6.2s
[CV 3/5] END ..................................., score=0.999 total time=   6.3s
[CV 4/5] END ..................................., score=0.999 total time=   6.3s


Let's see how the training goes if we remove `'plus_minus_home'`, `'pts_home'`, and `'pts_away'`. We expect the model to be able to predict the winner solely based on these. Since we will be dropping them before we select features using correlation or `SelectFromModel`, we expect this to have an impact on the models. If you only remove `'plus_minus_home'`, this impacts the performance, but barely. The models fail to be perfect, however, the worst scoring models, ExtraTrees and AdaBoost, have a lowest score of 97%.

In [ ]:
NEW_PIPELINES = create_pipelines()
'''fitted_pipelines = {}
for pipe_name, pipe in NEW_PIPELINES.items():
    pipe.fit(X_TrainSet,Y_TrainSet)
    fitted_pipelines[pipe_name] = pipe
'''
GRIDS_wo_pm = grid_search(X_TrainSet,Y_TrainSet,NEW_PIPELINES)

In [ ]:
def get_best_scores(GRIDS):
  for grid in GRIDS.values():
    res = (pd.DataFrame(grid.cv_results_)
       .sort_values(by='mean_test_score',ascending=False)
       .filter(['params','mean_test_score'])
       .values)

    print(res)

from sklearn.metrics import classification_report, confusion_matrix

def confusion_matrix_and_report(X,y,pipeline,label_map):

  prediction = pipeline.predict(X)

  print('---  Confusion Matrix  ---')
  print(pd.DataFrame(confusion_matrix(y_pred=prediction, y_true=y),
        columns=[ ["Actual " + sub for sub in label_map] ], 
        index= [ ["Prediction " + sub for sub in label_map ]]
        ))
  print("\n")


  print('---  Classification Report  ---')
  print(classification_report(y, prediction, target_names=label_map),"\n")


def clf_performance(X_train,y_train,X_test,y_test,pipeline,label_map):
  print("#### Train Set #### \n")
  confusion_matrix_and_report(X_train,y_train,pipeline,label_map)

  print("#### Test Set ####\n")
  confusion_matrix_and_report(X_test,y_test,pipeline,label_map)

 

After examining the reports for all of the models in `GRIDS` and those in our new search `GRIDS_wo_pm`, we found that the two worst performing models are still very close to being 100% accurate. They are AdaBoost and ExtraTrees. Other models did fail to be perfect, but these were the only two that did not score 100% across the board on all metrics for the test set.

In [ ]:
for name, grid in GRIDS_wo_pm.items():
    print(name)
    best_estimator = grid.best_estimator_
    clf_performance(X_TrainSet,Y_TrainSet,X_TestSet,Y_TestSet,best_estimator, label_map=['loss', 'win']) 

In [ ]:
best_ada = GRIDS_wo_pm['AdaBoost'].best_estimator_
best_extra_tree = GRIDS_wo_pm['ExtraTrees'].best_estimator_
print("Ada Boost")
clf_performance(X_TrainSet,Y_TrainSet,X_TestSet,Y_TestSet,best_ada, label_map=['loss', 'win'])

print("Extra Tree")
clf_performance(X_TrainSet,Y_TrainSet,X_TestSet,Y_TestSet, best_extra_tree, label_map=['loss', 'win'])

Let's inspect which features our models used.

In [ ]:
for pipe_name, pipe in NEW_PIPELINES.items():
    print(pipe_name)
    print(find_features(X_TrainSet,Y_TrainSet,pipe))

Section 1 content

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
